In [ ]:
#This performs MH algorithm on a partially-observed birth-death process 0->X->0 with birth parameter A and death parameter B.
import pandas as pd
import numpy as np
from scipy.stats import gamma
from scipy.stats import poisson

In [ ]:
#import saved data
pd_subsampled_x=pd.read_csv("subsampled.csv")
subsampled_x= pd_subsampled_x.values
x=np.reshape(subsampled_x,(1,len(subsampled_x)))
size=np.shape(x)

In [ ]:
#initializations
N=5000        #length of the chain
A=np.zeros(N)
B=np.zeros(N)
A[0]=1        #initialize the Markov Chain A
B[0]=1        #initialize the Markov Chain B
var_A=2       #set proposal variance for A
var_B=0.05    #set proposal variance for B
Ashape=4.5    #set the shape of the Gamma prior for A
Ascale=1      #set the scale of the Gamma prior for A
Bshape=0.25   #set the shape of the Gamma prior for B
Bscale=1      #set the scale of the Gamma prior for B
t=range(size[1])

In [ ]:
def init_reaction(x,s):
    q=np.zeros([2,size[1]-1])
    for i in range(s-1):
        if x[0,i+1]-x[0,i]>0:
            q[0,i]=np.floor((4/3)*(x[0,i+1]-x[0,i]))
            q[1,i]=np.floor((1/3)*(x[0,i+1]-x[0,i]))
        else:
            q[0,i]=-np.floor((1/3)*(x[0,i+1]-x[0,i]))
            q[1,i]=-np.floor((4/3)*(x[0,i+1]-x[0,i]))
    r=q.astype(int)        
    return r  

def hom_poisson_process(r,x,A,B,size,r_dim):
    t_birth=np.zeros([r_dim[0],size-1])
    t_death=np.zeros([r_dim[1],size-1])
    for j in range(size-1):
        if r[0,j]>0:
            t_birth[:r[0,j],j]=j+np.sort(np.random.uniform(0,1,r[0,j]))
        h_k1=B*x[0,j]
        h_k2=B*x[0,j+1]
        if r[1,j]>0:       
            t_death[:r[1,j],j]=np.sort(np.random.uniform(0,1,r[1,j]))
            if x[0,j]!=x[0,j+1]:
                for p in range(r[1,j]):
                    t_death[p,j]=j+((np.sqrt(h_k1**2+(h_k2**2-h_k1**2)*t_death[p,j])-h_k1)/(h_k2-h_k1))
            else:
                t_death[p,j]+=j
    return [t_birth, t_death]

def loglikelihood_x(z,r,t_b,t_d,A,B,size):
    size1=np.shape(z)
    like=np.zeros(size-1)
    integral=np.zeros((size-1))
    for j in range(size-1):
        a=np.zeros([r[0,j],1])
        b=np.zeros([r[1,j],1])
        w=np.zeros([r[0,j]+r[1,j],1])
        y=np.zeros([2+r[0,j]+r[1,j],1])
        h0_vector=None
        y[0,0]=z[0,j]
        y[len(y)-1,0]=z[0,j+1]
        for i in range(r[0,j]):
            a[i,0]=t_b[i,j]
        for i in range(r[1,j]):
            b[i,0]=t_d[i,j]
        m=np.sort(np.ndarray.flatten(np.append(a,b, axis=0)))
        for k in range(len(m)):
            if np.any(a==m[k])==True:
                y[k+1,0]=y[k,0]+1
                w[k,0]=np.log(A)
            else:
                y[k+1,0]=y[k,0]-1
                w[k,0]=np.log(B*y[k-1,0])
        y=np.ndarray.flatten(y)        
        like[j]=np.sum(w)
        h0_vector=A+B*y    
    loglik=np.sum(like)-np.trapz(A+B*np.ndarray.flatten(z),np.arange(size1[1]))
    return loglik

def accept_rate_react(x,r,prop,i,A,B):
    prop_like=poisson.pmf(prop[0],A)*poisson.pmf(prop[1],0.5*B*(x[0,i]+x[0,i+1]))
    current_like=poisson.pmf(r[0],A)*poisson.pmf(r[1],0.5*B*(x[0,i]+x[0,i+1]))  
    rate=np.minimum(1,prop_like/current_like)
    return rate
 
def accept_rate_x(var,x,r,t_b,t_d,A,B,size,prop,shape,scale1):
    if var=='birth':
        prop_like=loglikelihood_x(x,r,t_b,t_d,prop,B,size)+np.log(gamma.pdf(prop,a=shape,scale=scale1))
        current_like=loglikelihood_x(x,r,t_b,t_d,A,B,size)+np.log(gamma.pdf(A,a=shape,scale=scale1))
    else:
        prop_like=loglikelihood_x (x,r,t_b,t_d,A,prop,size) +np.log(gamma.pdf(prop,a=shape,scale=scale1))
        current_like=loglikelihood_x(x,r,t_b,t_d,A,B,size) +np.log(gamma.pdf(B,a=shape,scale=scale1))
    rate=np.minimum(1,np.exp(prop_like-current_like))
    return rate

In [ ]:
#Initialize reaction numbers and times
r=init_reaction(x,size[1])

#MCMC
for i in range(N-1):
    
    #Perform approximate block updating
    for j in range(size[1]-1):   
        r_vprop=[-1,-1]
        while r_vprop[0]<0 or r_vprop[1]<0: 
            jump=np.random.poisson(2)-np.random.poisson(2)
            r_prop=r[0,j]+jump
            r_vprop=[r_prop, r_prop-(x[0,j+1]-x[0,j])]
        rate=accept_rate_react(x,r[:,j],r_vprop,j,A[i],B[i])
        if np.random.uniform(0,1,1)<rate:
            r[:,j]=r_vprop      
    r_dim=np.max(r,axis=1).astype(int)        
    [t_birth,t_death]=hom_poisson_process(r,x,A[i],B[i],size[1],r_dim)     
    
    #Perform Gibbs sampling
    if i % 2 != 0:     #MH for A
        propA=-1
        while propA<0:
            propA=A[i]+np.random.normal(0,var_A,1)
        rate=accept_rate_x('birth',x,r,t_birth,t_death,A[i],B[i],size[1],propA,Ashape,Ascale)
        if np.random.uniform(0,1,1)<rate:
            A[i+1]=propA
            B[i+1]=B[i]
        else:
            A[i+1]=A[i]
            B[i+1]=B[i]
    else:              #MH for B
        propB=-1
        while propB<0:
            propB=B[i]+np.random.normal(0,var_B,1)
        rate=accept_rate_x('death',x,r,t_birth,t_death,A[i],B[i],size[1],propB,Bshape,Bscale)
        if np.random.uniform(0,1,1)<rate:
            B[i+1]=propB
            A[i+1]=A[i]
        else:
            B[i+1]=B[i]
            A[i+1]=A[i]
    if i%50==0:
        print([A[i+1],B[i+1]])